<a href="https://colab.research.google.com/github/Maria-Evlakhova/ClinicalRecommendations/blob/main/%D0%91%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82_%22%D0%9A%D0%BB%D0%B8%D0%BD%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%B8%D0%B5_%D1%80%D0%B5%D0%BA%D0%BE%D0%BC%D0%B5%D0%BD%D0%B4%D0%B0%D1%86%D0%B8%D0%B8_ipynb%22%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Создание вопросно-ответной системы
## Что из себя представляют вопросно-ответные системы с использованием языковых моделей


Инструменты:
- `gigachain` - основной инструмент построения пайплайнов с большими языковыми моделями;
- `faiss-cpu` - используем векторное хранилище FAISS для размещения чанков (фрагментов текстов для контекста)и эмбеддингов (векторных представлений) этих чанков;
- `sentence-transformers` - библиотека для локального использования эмбеддинговых моделей с ресурса Hugging Face;
- `rank_bm25` - библиотека для работы с алгоритмами векторизации BM25 (вариации TF-IDF);
- `playwright` - библиотека для автоматизации браузера, в нашем случае для парсинга списка статей с веб-сайта.

In [7]:
!pip install gigachain faiss-cpu sentence-transformers rank_bm25 playwright langchain-community unstructured[all-docs] pdf2image --quiet
!pip install -U langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 43.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 9.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 41.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 37.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 MB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 46.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 403.7/403.7 kB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.8/311.8 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

установка браузера для сбора списка статей.

In [8]:
!playwright install

164.9 MiB [] 0% 0.0s164.9 MiB [] 0% 7.8s164.9 MiB [] 0% 6.8s164.9 MiB [] 0% 5.2s164.9 MiB [] 1% 5.4s164.9 MiB [] 1% 6.6s164.9 MiB [] 1% 6.7s164.9 MiB [] 2% 6.9s164.9 MiB [] 2% 7.2s164.9 MiB [] 2% 7.0s164.9 MiB [] 2% 6.6s164.9 MiB [] 3% 6.6s164.9 MiB [] 3% 6.5s164.9 MiB [] 3% 6.6s164.9 MiB [] 4% 6.5s164.9 MiB [] 4% 6.3s164.9 MiB [] 5% 5.9s164.9 MiB [] 5% 5.6s164.9 MiB [] 6% 5.4s164.9 MiB [] 6% 5.8s164.9 MiB [] 6% 5.9s164.9 MiB [] 6% 6.2s164.9 MiB [] 6% 6.5s164.9 MiB [] 7% 7.0s164.9 MiB [] 7% 7.3s164.9 MiB [] 7% 7.2s164.9 MiB [] 8% 9.4s164.9 MiB [] 8% 9.3s164.9 MiB [] 9% 9.2s164.9 MiB [] 9% 9.0s164.9 MiB [] 10% 8.8s164.9 MiB [] 10% 8.4s164.9 MiB [] 11% 8.2s164.9 MiB [] 11% 8.1s164.9 MiB [] 11% 8.2s164.9 MiB [] 12% 8.6s164.9 MiB [] 12% 8.7s164.9 MiB [] 12% 8.5s164.9 MiB [] 13% 8.4s164.9 MiB [] 14% 7.9s164.9 MiB [] 14% 8.3s164.9 MiB [] 14% 8.2s164.9 MiB [] 15% 8.8s164.9 MiB [] 15% 9.0s164.9 MiB [] 16% 8.9s164.9 MiB [] 16% 8.8s164.9 MiB [] 16% 8.6s164.9 MiB [] 17% 8.7s164.9 MiB [] 17% 8.6s1

In [9]:
import os

# Словари для хранения диагнозов и кодов МКБ-10
diagnoses = {
    "Грипп": "J10",
    "Пневмония": "J18",
    "Диабет": "E11"
}

icd_codes = {
    "J10": "грипп",
    "J18": "пневмония",
    "E11": "диабет"
}

# Словарь для связывания кодов МКБ с PDF-файлами
pdf_files = {
    "J10": "грипп.pdf",
    "J18": "пневмония.pdf",
    "E11": "диабет.pdf"
}

def get_icd_code(diagnosis):
    return diagnoses.get(diagnosis)

def main():
    # Запрос диагноза у пользователя
    diagnosis = input("Введите диагноз: ")

    # Получение кода МКБ-10
    icd_code = get_icd_code(diagnosis)

    if icd_code:
        print(f"Код МКБ-10 для '{diagnosis}': {icd_code}")
        confirmation = input("Подтверждаете? (да/нет): ").strip().lower()

        if confirmation == "да":
            # Проверка наличия PDF-файла
            pdf_file = pdf_files.get(icd_code)
            if pdf_file and os.path.exists(pdf_file):
                print(f"Загружаем файл: {pdf_file}")
                # Здесь можно добавить код для открытия PDF-файла
            else:
                print("PDF-файл не найден.")
        else:
            print("Введите другой диагноз.")
            # Можно добавить логику для поиска другого кода МКБ-10
    else:
        print("Диагноз не найден.")

if __name__ == "__main__":
    main()

Введите диагноз: Грипп
Код МКБ-10 для 'Грипп': J10
Подтверждаете? (да/нет): да
PDF-файл не найден.


### Сбор информации о клинических рекомендациях


In [10]:
%%writefile parser.py

from langchain_community.document_loaders import AsyncChromiumLoader
from bs4 import BeautifulSoup

# Инициализируем загрузчик
loader = AsyncChromiumLoader(["https://org.gnicpm.ru/pomosh/klinicheskie-rekomendaczii.html"])
html = loader.load()
articles = 'download'
with open('page.html', 'w') as f:
  f.write(html[0].page_content)
soup = BeautifulSoup(html[0].page_content, 'html.parser')
links = []
with open('links.txt', 'w') as f:
  for link in soup.find_all('a', class_=articles):
    print(link)
    links.append(link['href'])
    f.write(link['href']+' ')


Writing parser.py


In [11]:
!python parser.py

USER_AGENT environment variable not set, consider setting it to identify your requests.
<a class="download" href="https://org.gnicpm.ru/wp-content/uploads/2024/01/323-fz.pdf">Федеральный закон от 21.11.2011 № 323-ФЗ (ред. от 25.12.2023) «Об основах охраны здоровья граждан в Российской Федерации»</a>
<a class="download" href="https://org.gnicpm.ru/wp-content/uploads/2024/04/pp-rf-ot-17.11.2021-№1968.pdf">Постановление Правительства РФ от 17.11.2021 № 1968 (ред. от 14.03.2024) «Об утверждении Правил поэтапного перехода медицинских организаций к оказанию медицинской помощи на основе клинических рекомендаций, разработанных и утвержденных в соответствии с частями 3, 4, 6 — 9 и 11 статьи 37 Федерального закона «Об основах охраны здоровья граждан в Российской Федерации»</a>
<a class="download" href="https://org.gnicpm.ru/wp-content/uploads/2024/05/postanovlenie-pravitelstva-rf-ot-28_12_2023-n-2353-v-redakczii-ot-23.03.2024g.pdf">Постановление Правительства Российской Федерации от 28.12.2023 №

## Подготовка фрагментов статей, векторизация и загрузка в векторную БД

Откроем файл со ссылками.

In [12]:

with open('links.txt') as f:
  sources = f.read()

print(sources.split())


['https://org.gnicpm.ru/wp-content/uploads/2024/01/323-fz.pdf', 'https://org.gnicpm.ru/wp-content/uploads/2024/04/pp-rf-ot-17.11.2021-№1968.pdf', 'https://org.gnicpm.ru/wp-content/uploads/2024/05/postanovlenie-pravitelstva-rf-ot-28_12_2023-n-2353-v-redakczii-ot-23.03.2024g.pdf', 'https://org.gnicpm.ru/wp-content/uploads/2024/04/pp-ot-14.03.2024-№299.pdf', 'https://cr.minzdrav.gov.ru/preview-cr/62_3', 'https://cr.minzdrav.gov.ru/preview-cr/155_2', 'https://cr.minzdrav.gov.ru/preview-cr/160_1', 'https://cr.minzdrav.gov.ru/preview-cr/382_1', 'https://cr.minzdrav.gov.ru/preview-cr/619_2', 'https://cr.minzdrav.gov.ru/preview-cr/569_1', 'https://cr.minzdrav.gov.ru/preview-cr/156_2', 'https://cr.minzdrav.gov.ru/preview-cr/752_1', 'https://cr.minzdrav.gov.ru/preview-cr/290_2', 'https://cr.minzdrav.gov.ru/preview-cr/277_1', 'https://cr.minzdrav.gov.ru/preview-cr/273_4', 'https://cr.minzdrav.gov.ru/preview-cr/655_1', 'https://cr.minzdrav.gov.ru/preview-cr/603_2', 'https://cr.minzdrav.gov.ru/prev

Импортируем нужные инструменты.


In [13]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain.vectorstores.faiss import FAISS
from langchain_core.documents import Document

/usr/local/lib/python3.11/dist-packages/pydantic/v1/validators.py:767: UserWarning: Mixing V1 and V2 models is not supported. `ChatGeneration` is a V2 model.
  warn(f'Mixing V1 and V2 models is not supported. `{type_.__name__}` is a V2 model.', UserWarning)


RuntimeError: no validator found for <class 'langchain_core.outputs.chat_generation.ChatGeneration'>, see `arbitrary_types_allowed` in Config

In [ ]:
import os
os.environ['USER_AGENT'] = 'myagent'
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

Осуществим загрузку веб-страниц по ссылкам.

In [ ]:
loader = WebBaseLoader(sources.split())

In [ ]:
docs = loader.load()

In [ ]:
len(docs)

30

Разделим тексты статей на фрагменты длиной 500 символов и перекрытием в 100 символов.

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500,
                                              chunk_overlap=100)
split_docs = text_splitter.split_documents(docs)

In [ ]:
len(split_docs)

19038

In [ ]:
split_docs[1]

Document(metadata={'source': 'https://org.gnicpm.ru/wp-content/uploads/2024/01/323-fz.pdf'}, page_content='xœ�}\x18Ä6Ny¯ìØ¥Ö\x03¢¤\x11˜.ã1¶ÃĞ\x0bv¨Àã$Ü\x06à+Øª¯Ø30��Ju=»\x014\xa0×Ü¦)Å"s”\xadj\'|QêÓƒdÒªU\x1aµZR?\x0b…á!h[…ÅNãeŸ·ÌãÁ²<9�i³°\x0eÕzÆîò0\x16öä¡\x11µj\x0eğ+…T}¤JRodı\x01\x1cQ•ÃV8KĞæÃ-§²ş\x11iqâ’`d\x11&\x06Ó•�ùä*GœbJ„æq±\x1eŸ†}\x08fÑ‰f–ÈLØ¡fu4�QO\x12\x04A\x10\x0411Œá)õl`’]\x0eA\x10g"ü\x1bp¸ƒn†wõaĞƒ>ü!\x18À€j\x04#ª\tL¨f°„?\x00‹P+ØÂïƒ\rì¨vp\xa0: \x015\x01\x12Q\x13!\tÕ\tÉ¨I�‚š,4\x05RÃïA*¤‡�AšĞtp¡º \x135\x03²P3…f�\x07Õ\r^T\x0fê¿À\x0bÙ¨Ù0\x05Õ\x079¨S \x175\x07¦†ß…\\ÈC�*4\x0fòÃÿ„iàGÍ‡é¨~(@�\x0e…áw\xa0@h!\x14£\x16Á\x0cÔb(A�\x01¥¨%P†Z\nå¨e¨oÃL¨@-‡JÔ\n¡•P\x1d~\x0bª\xa0\x06u\x16ÌF\xad\x16Z\x03ç\xa0Î†ZÔs \x10ş?¨…9á7!\x00õ¨s„ÖA\x03j=4¡6À\\ÔF¡M0\x0fu.œ‹:\x0f\x16„ÿ\x01óaaø\r8\x17ÎC] t!œ�z\x1e,B=\x1f–\xa0.‚¥¨‹a\x19ê\x12X\x1e~\x1d–')

In [5]:
!pip install numpy==1.26.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.5/58.5 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 80.9 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.4
    Uninstalling numpy-2.2.4:
      Successfully uninstalled numpy-2.2.4


Создадим ретривер и векторное хранилище для врагментов текста.


In [ ]:
%%time
model_name = "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
embedding = HuggingFaceEmbeddings(model_name=model_name,
                                  model_kwargs=model_kwargs,
                                  encode_kwargs=encode_kwargs)

vector_store = FAISS.from_documents(split_docs, embedding=embedding)

NameError: name 'HuggingFaceEmbeddings' is not defined

Зададим параметры извлечения, на запрос должны возвращаться 5 фрагментов, наиболее близких по смыслу.

In [ ]:
embedding_retriever = vector_store.as_retriever(search_kwargs={"k": 5})

## Создание генеративной части пайплайна и составление цепочки RAG системы

Используем авторизационные данные для подключения к GigaChat API.

In [ ]:
from google.colab import userdata
auth = userdata.get('SBER_AUTH')

Импортируем необходимые компоненты

In [ ]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chat_models.gigachat import GigaChat
from langchain.chains import create_retrieval_chain

Создадим объект `GigaChat` и подготовим промпт для вопросно-ответной системы.

In [ ]:

llm = GigaChat(credentials=auth,
              model='GigaChat:latest',
               verify_ssl_certs=False,
               profanity_check=False)
prompt = ChatPromptTemplate.from_template('''Ответь на вопрос пользователя. \
Используй при этом только информацию из контекста. Если в контексте нет \
информации для ответа, сообщи об этом пользователю.
Контекст: {context}
Вопрос: {input}
Ответ:'''
)

<ipython-input-54-7895cbad50b3>:1: LangChainDeprecationWarning: The class `GigaChat` was deprecated in LangChain 0.3.5 and will be removed in 1.0. An updated version of the class exists in the langchain-gigachat package and should be used instead. To use it run `pip install -U langchain-gigachat` and import as `from langchain_gigachat import GigaChat`.
  llm = GigaChat(credentials=auth,


Создадим цепочку `create_stuff_documents_chain`, которая будет частью нашей вопросно-ответной цепочки. Это нужно, чтобы подавать фрагменты текстов из векторной БД в промпт языковой модели. Промпт представляет из себя форматированную строку, а франменты являются экземплярами класса `Document`. Чтобы не писать код по извлечению атрибута `page_content` из `Document`, используем цепочку `create_stuff_documents_chain`, где это автоматизировано.

In [ ]:
document_chain = create_stuff_documents_chain(
    llm=llm,
    prompt=prompt
    )

А теперь создадим вопросно-ответную цепочку с помощью функции `create_retrieval_chain()`.

In [ ]:
retrieval_chain = create_retrieval_chain(embedding_retriever, document_chain)

## Запустим нашу вопросно-ответную систему


In [ ]:
q1 = 'Грипп как диагностировать'

In [ ]:
resp1 = retrieval_chain.invoke(
    {'input': q1}
)

In [ ]:
resp1

{'input': 'Сахарный диабет 2 типа какой код мкб',
 'context': [Document(id='cea6af13-be1c-466a-a252-c1e26027a78c', metadata={'source': 'https://medi.ru/filters/doc_type/manuals/oftalmologiya/', 'title': 'Офтальмология', 'description': 'Клинические рекомендации МЗ РФ…', 'language': 'ru'}, page_content='Травма глаза закрытая\n\nКлинические рекомендации\n\n\n\t\t\t10 окт. 2016 г..\n\t\t\n\n читать..\n\n\n\n\n\n\n\nСахарный диабет: диабетическая ретинопатия, диабетический макулярный отек\n\nКлинические рекомендации\n\n\n\t\t\t10 окт. 2016 г..\n\t\t\n\n читать..\n\n\n\n\n\n\n\nРетинопатия недоношенных, активная фаза\n\nКлинические рекомендации\n\n\n\t\t\t10 окт. 2016 г..\n\t\t\n\n читать..\n\n\n\n\n\n\n\nРетинобластома у детей\n\nКлинические рекомендации\n\n\n\t\t\t10 окт. 2016 г..\n\t\t\n\n читать..\n\n\n\n\n\n\n\nРегматогенная отслойка сетчатки\n\nКлинические рекомендации'),
  Document(id='e164466d-130c-4fc5-a168-bad84616acea', metadata={'source': 'https://medi.ru/filters/doc_type/manuals

In [1]:
q2 = 'Как лечить грипп?'

In [2]:
resp2 = retrieval_chain.invoke(
    {'input': q2}
)

NameError: name 'retrieval_chain' is not defined

In [3]:
resp2

NameError: name 'resp2' is not defined